In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from gelos.gelosdataset import GELOSDataSet
from gelos.gelosdatamodule import GELOSDataModule


2025-10-17 20:11:51.356 | INFO     | gelos.config:<module>:11 - PROJ_ROOT path is: /app
/opt/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.8 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations
/opt/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a sing

In [3]:
from lightning.pytorch import Trainer

In [4]:
import yaml

In [5]:
data_root = Path('/app/data/raw/v0.40')

In [6]:
gelos_datamodule = GELOSDataModule(data_root=data_root, batch_size=8, num_workers=16)

In [7]:
output_dir = Path('/app/data/interim/v0.40')
output_dir.mkdir(exist_ok=True)

In [8]:
config_file = "/app/gelos/configs/prithvi_eo_300m_embedding_generation.yaml"

In [9]:
with open(config_file, 'r') as file:
    config = yaml.safe_load(file)

In [10]:
print(yaml.dump(config))

data:
  class_path: gelos.gelosdatamodule.GELOSDataModule
  init_args:
    bands:
      S2L2A:
      - BLUE
      - GREEN
      - RED
      - RED_EDGE_1
      - NIR_BROAD
      - NIR_NARROW
    batch_size: 1
    data_root: /app/data/raw/v0.40
    num_workers: 0
model:
  class_path: terratorch.tasks.EmbeddingGenerationTask
  init_args:
    embed_file_key: filename
    embedding_pooling: null
    has_cls: true
    layers:
    - -1
    model: prithvi_eo_v2_300
    model_args:
      backbone: prithvi_eo_v2_300
      backbone_bands:
      - BLUE
      - GREEN
      - RED
      - RED_EDGE_1
      - NIR_BROAD
      - NIR_NARROW
      backbone_pretrained: true
    output_dir: /app/data/interim/v0.40
    output_format: parquet
predict_output_dir: ./
seed_everything: 0
trainer:
  accelerator: auto
  callbacks: []
  devices: auto
  max_epochs: 0
  num_nodes: 1
  strategy: auto



In [11]:
gelos_datamodule = GELOSDataModule(**config['data']['init_args'])

In [12]:
from terratorch.tasks import EmbeddingGenerationTask

In [13]:
task = EmbeddingGenerationTask(**config['model']['init_args'])

/opt/venv/lib/python3.11/site-packages/terratorch/tasks/embedding_generation.py:67: UserWarning: GeoTIFF selected; 2D token embeddings (ViT) will be reshaped to [C, sqrt(num_tokens), sqrt(num_tokens)] after dropping CLS if present.
  warnings.warn(
INFO:terratorch.models.backbones.prithvi_vit:model_bands not passed. Assuming bands are ordered in the same way as [<HLSBands.BLUE: 'BLUE'>, <HLSBands.GREEN: 'GREEN'>, <HLSBands.RED: 'RED'>, <HLSBands.NIR_NARROW: 'NIR_NARROW'>, <HLSBands.SWIR_1: 'SWIR_1'>, <HLSBands.SWIR_2: 'SWIR_2'>].Pretrained patch_embed layer may be misaligned with current bands


In [14]:
trainer = Trainer(accelerator='cpu')

INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/opt/venv/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [15]:
trainer.predict(model=task, datamodule=gelos_datamodule)

/opt/venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Predicting DataLoader 0:   0%|          | 1/77547 [00:00<7:39:00,  2.82it/s]

/opt/venv/lib/python3.11/site-packages/lightning/pytorch/loops/prediction_loop.py:257: predict returned None if it was on purpose, ignore this warning...


Predicting DataLoader 0:   0%|          | 23/77547 [00:05<4:41:05,  4.60it/s]

INFO: 
Detected KeyboardInterrupt, attempting graceful shutdown ...
INFO:lightning.pytorch.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined